In [6]:
import tornado.ioloop

In [7]:
i = 0

In [3]:
def cb0():
    global i
    i = i + 1
    print('cb0-call#{:02d}'.format(i))

In [4]:
pcb0 = tornado.ioloop.PeriodicCallback(cb0, 1000.)

In [5]:
pcb0.start()

cb0-call#01
cb0-call#02


In [6]:
# another cell
print("this is the 'current' cell!")

this is the 'current' cell!
cb0-call#03
cb0-call#04


In [7]:
# another cell
print("this is the 'current' cell!")

this is the 'current' cell!
cb0-call#05
cb0-call#06


In [8]:
pcb0.stop()

--

In [9]:
import sys
from IPython.display import clear_output

class CellOutput(object):
    def __init__(self):
        k = get_ipython().kernel
        self._ident = k._parent_ident
        self._header = k._parent_header
        self._save_context = None

    def __enter__(self):
        kernel = get_ipython().kernel
        self._save_context = (kernel._parent_ident, kernel._parent_header)
        sys.stdout.flush()
        sys.stderr.flush()
        kernel.set_parent(self._ident, self._header)

    def __exit__(self, etype, evalue, tb):
        sys.stdout.flush()
        sys.stderr.flush()
        kernel = get_ipython().kernel
        kernel.set_parent(*self._save_context)
        return False

    def clear_output(self):
        with self:
            clear_output()

In [10]:
out1 = CellOutput()

In [8]:
def cb1():
    global i
    i = i + 1
    out1.clear_output()
    with out1:
        if not i % 10:
            return 1.0 / 0.0
        print('cb1-call-#{:02d}'.format(i))

In [9]:
pcb1 = tornado.ioloop.PeriodicCallback(cb1, 1000.)

In [10]:
pcb1.start()

In [11]:
# another cell
print("this is the 'current' cell!")

this is the 'current' cell!


In [12]:
# another cell
print("this is the 'current' cell!")

this is the 'current' cell!


In [13]:
pcb1.stop()

In [14]:
out1.clear_output()

NameError: name 'out1' is not defined

--

In [48]:
i = 1

In [49]:
import ipywidgets
out2 = ipywidgets.Output()
ftxt = ipywidgets.IntText(value=i, description='i-value:', disabled=True)
display(ftxt)
display(out2)

IntText(value=1, description='i-value:', disabled=True)

Output()

In [50]:
out2.outputs

()

In [51]:
def cb2():
    global i
    i = i + 1
    try:
        if not i % 10:
            return 1.0 / 0.0
        else:
            ftxt.value = i
    except Exception as e:
        with out2:
            print(e)
        raise(e)
    out2.outputs = ()

In [52]:
out2.outputs

()

In [53]:
pcb2 = tornado.ioloop.PeriodicCallback(cb2, 500.)

In [54]:
pcb2.start()

In [55]:
# another cell
print("this is the 'current' cell!")

this is the 'current' cell!


In [56]:
# another cell
print("this is the 'current' cell!")

this is the 'current' cell!


In [57]:
pcb2.stop()

In [58]:
out2.clear_output()

In [100]:
debug_view = ipywidgets.Output(layout={'border': '1px solid black'})
display(debug_view)

#@debug_view.capture(clear_output=True)
def bad_callback(event):
    print('This is about to explode')
    return 1.0 / 0.0

button = ipywidgets.Button(
    description='click me to raise an exception',
    layout={'width': '300px'}
)
button.on_click(bad_callback)
button

Output(layout=Layout(border='1px solid black'))

Button(description='click me to raise an exception', layout=Layout(width='300px'), style=ButtonStyle())

--